## 1. Change the directory

In [ ]:
%cd /content/drive/MyDrive/recipekogpt2

/content/drive/.shortcut-targets-by-id/1jOKmNyG5BEUAKKmGZ__qNyXG3qcjgYHB/recipekogpt2


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 2. Load and check the data structure

데이터는 딕셔너리 형태로 저장되어 있으며, 재료부분은 value값이 리스트 형태입니다.

In [ ]:
import pickle
import pandas as pd

with open('/content/drive/MyDrive/data/all_token_pkl.pkl', "rb") as fr:
    dfp = pickle.load(fr)
df = pd.DataFrame(dfp)
df.rename(columns={'음식명':'title','재료':'ingredients','레시피':'instructions'}, inplace = True)
df_t = df.loc[:,['title','ingredients','instructions']]
df_t.shape

(22594, 3)

In [ ]:
df_t.isnull().any()

title           False
ingredients     False
instructions     True
dtype: bool

In [ ]:
df_drop = df_t.dropna(axis=0)

In [ ]:
data = df_drop.to_dict('records')
data[0:3]

[{'ingredients': ['고구마', '설탕', '찹쌀', '가루', '물', '잣'],
  'instructions': '고구마는 깨끗이 씻어서 껍질을 벗기고 4 정도로 잘라준다. 찜기에 고구마를 넣고 20-30분 정도 삶아 주고 블렌더나 체를 이용하여 잘 으 깨어 곱게 만든다. 고구마와 물을 섞어 끓이면서 찹쌀가루로 농도를 맞추고 설탕을 넣어 맛을 낸다. 잣을 팬에 노릇하게 볶아 다져서 고구마 죽에 섞는다. 기호에 따라 고구마를 튀겨 얹어 먹어도 좋다.',
  'title': '칼륨 듬뿍 고구마죽'},
 {'ingredients': ['채소',
   '애호박',
   '표고버섯',
   '당근',
   '누룽지',
   '순두부',
   '달걀',
   '참기름',
   '소금',
   '참깨',
   '후추'],
  'instructions': '깨끗이 씻어 손질한 애호박 당근과 기둥을 뗀 표고버섯을 잘게 다지듯이 썬다. 누룽지는 1 정도로 잘게 부숴준다. 냄비에 참기름을 두르고 썰어 놓은 채소를 볶다가 누룽지와 물을 넣고 끓인다. 누룽지가 살짝 퍼지면 순두부를 넣고 흰 후추와 소금을 넣는다. 죽이 끓으면 달걀을 풀어 넣고 한 소끔 끓여낸 후 참깨를 뿌려 마무리한다.',
  'title': '누룽지 두부 계란죽'},
 {'ingredients': ['초밥',
   '밥',
   '공기',
   '배합',
   '초',
   '식초',
   '설탕',
   '소금',
   '오색',
   '지라시',
   '달걀노른자',
   '달걀',
   '표고버섯',
   '장',
   '송이버섯',
   '새우',
   '마리',
   '홍',
   '피망',
   '청',
   '양파',
   '두유',
   '참기름',
   '식용유'],
  'instructions': '배합초는 중불에서 설탕이 녹을 때까지 저어가면서 녹인다. 뜨거운 밥에 배합초를 넣어서 밥알이 으깨지지 않게 고루 저어가면서 밥을 체온 정도로 식힌다. 표고버섯은 기둥을 떼고 끓는 물에 

## 3. Change the data structure

딕셔너리 형태의 데이터를 txt형태로 바꿔줍니다. 이 때, 각 부분(ingredients, title, instructions)의 시작과 끝 부분에 토큰을 붙여줍니다. 토큰은 추후 GPT-2가 훈련할 때 각 부분이 시작하고 끝난다는 것을 나타내줍니다. 또한 각 재료 사이에는 '$'문자를 삽입해 재료를 구분해줍니다.

In [ ]:
from preprocessing.dependency import parent_dir
from preprocessing.common.basics import *
from preprocessing.common.save import make_dir, save_pickle, load_pickle, save
import random
import tqdm

In [ ]:
def txt(v, fields, mode = 'train'):
    """ transform the recipe dict to string
    
        Args:
          v: A dict of recipe information
          fields: an order list, the last is the field to predict
          mode: string

        Returns:
          In test mode, return string X, y
          In train mode, return string X + y
    """
    to_write = ''
    for field in fields:
        if field == 'title':
            name = v['title']
            to_write += '<unused0>'+name+'<unused1>'
        if field == 'ingredients':
            ingredients = v['ingredients']
            to_write += '<unused2>'+'$'.join(ingredients)+'$<unused3>'
        if field == 'instructions':
            instructions = v['instructions']
            to_write += '<unused4>'+ instructions +'<unused5>'
    to_write += '\n'
            
    if mode == 'train':
        return to_write
                                                     
    elif mode == 'test':
        field_to_predict = '<start-%s>'%fields[-1]
        to_X, to_y = to_write.split(field_to_predict)
        return to_X+field_to_predict, to_y

class to_gpt2:
    def __init__(self, data, ls = None):
        if not ls:
            ls = list(range(len(data)))
        random.seed(42)
        random.shuffle(ls)
        self.ls_test = ls[:2173] #10, 10, 80, test, val, train
        self.ls_val = ls[2173:4346]
        self.ls_train = ls[4346:]
        self.data = data
        
    def test(self, ls, filename, overwrite = False, is_val = False):
        to_write = ''
        # order of field and the inital of the field to predict
        orders = [(['title','ingredients','instructions'], 'd'),
                  (['title','instructions','ingredients'], 'i'),
                  (['ingredients','instructions','title'], 't')]
        
        for i, v in tqdm.tqdm(enumerate(self.data)):
            if i in ls:
                for order, init in orders:
                    X, y = txt(v, order, mode = 'test')
                    self.save(filename+'X/%d'%(i)+'%s.txt'%(init), X, overwrite)
                    self.save(filename+'y/%d'%(i)+'%s.txt'%(init), y, overwrite)
        
    def save(self, filename, to_write, overwrite = False):
        make_dir(filename)
        if os.path.isfile(filename) == True and overwrite == False:
            print('already exists'+filename)
        else:    
            with open(filename,'w') as f:
                f.write('%s' % to_write)
                
    def make_txt(self, ls, filename, tag, overwrite = False):
        make_dir(filename)
        f = open(f'{filename}/{tag}', 'w')
        for i, v in tqdm.tqdm(enumerate(self.data)):
            if i in ls:
                f.writelines(self.encode_recipe(self.data[i]))      
        print(f'{filename}/{tag}')
        f.close()
        
    def encode_recipe(self,recipe):
        return txt(recipe, ['title','ingredients','instructions'])
    
    def fast_txt(self, filename, overwrite = False):
        self.make_txt(self.ls_train, filename, tag = 'recipegpt_train.txt')
        self.make_txt(self.ls_val, filename, tag = 'recipegpt_val.txt')
        self.make_txt(self.ls_test, filename, tag = 'recipegpt_test.txt')

클래스 선헌 후, 인스턴스를 만듭니다. 처리된 데이터를 저장할 디렉토리를 선언합니다.

In [ ]:
model = to_gpt2(data)
filename = './data/recipegpt'

In [ ]:
os.getcwd()

'/content/drive/.shortcut-targets-by-id/1jOKmNyG5BEUAKKmGZ__qNyXG3qcjgYHB/recipekogpt2'

In [ ]:
model.fast_txt(filename)

22590it [00:07, 3004.96it/s]
2217it [00:00, 22143.15it/s]

./data/recipegpt/recipegpt_train.txt


22590it [00:01, 19744.19it/s]
2286it [00:00, 22828.83it/s]

./data/recipegpt/recipegpt_val.txt


22590it [00:01, 19603.52it/s]

./data/recipegpt/recipegpt_test.txt
